In [ ]:
import os
import csv
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# helper functions

def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]

def unroll(series):
    l = []
    for key in series:
        if (key[-1] == 'v' or key[-1] == 'z'): continue
        l += (series[key])
    return l

def sparse_output(filename):
    o_index = int(filename[4])
    f_index = int(filename[7])
    b_index = int(filename[10])
    s_index = int(filename[13])
    l_index = int(filename[16])
    output = [0.0] * 50
    output[o_index] = 1
    output[f_index + 10] = 1.0
    output[b_index + 20] = 1.0
    output[s_index + 30] = 1.0
    output[l_index + 40] = 1.0
    return output

# TODO define sparse error functions

def sparseE0(predictions, truths):
    # returns vector of length 5 with E0 for one type of data processing
    o_num_correct = 0
    f_num_correct = 0
    b_num_correct = 0
    s_num_correct = 0
    l_num_correct = 0
    n = len(predictions)
    for i in range(len(predictions)):
        pred = predictions[i]

        truth = [int(j) for j in truths[i]]

        if (pred[truth[0]] > 0.5): # if overstride correct
            o_num_correct+=1
        if (pred[truth[1] + 10] > 0.5): # if forward lean correct
            f_num_correct+=1
        if (pred[truth[2] + 20] > 0.5): # if backward lean correct
            b_num_correct+=1 
        if (pred[truth[3] + 30] > 0.5): # if sweeping correct
            s_num_correct+=1
        if (pred[truth[4] + 40] > 0.5): # if low arms correct
            l_num_correct+=1
    return [float(o_num_correct) / n, float(f_num_correct) / n, float(b_num_correct) / n, float(s_num_correct) / n, float(l_num_correct) / n]



def sparseE1(predictions, truths):
    # returns vector of length 5 with E0 for one type of data processing
    o_num_correct = 0
    f_num_correct = 0
    b_num_correct = 0
    s_num_correct = 0
    l_num_correct = 0
    n = len(predictions)

    for i in range(len(predictions)):
        pred = predictions[i]

        truth = [int(j) for j in truths[i]]

        o_space = np.arange(max(truth[0] - 1, 0), min(truth[0] + 2, 9))
        f_space = np.arange(max(truth[1] + 10 - 1, 10), min(truth[1] + 10 + 2, 19))
        b_space = np.arange(max(truth[2] + 20 - 1, 20), min(truth[2] + 20 + 2, 29))
        s_space = np.arange(max(truth[3] + 30 - 1, 30), min(truth[3] + 30 + 2, 39))
        l_space = np.arange(max(truth[4] + 40 - 1, 40), min(truth[4] + 40 + 2, 49))

        for truth_index in o_space:
            if (pred[truth_index] > 0.5): # if overstride correct
                o_num_correct+=1
                break

        for truth_index in f_space:
            if (pred[truth_index] > 0.5): # if forward lean correct
                f_num_correct+=1
                break
        
        for truth_index in b_space:
            if (pred[truth_index] > 0.5): # if backward lean correct
                b_num_correct+=1
                break

        for truth_index in s_space:
            if (pred[truth_index] > 0.5): # if sweeping correct
                s_num_correct+=1
                break

        for truth_index in l_space:
            if (pred[truth_index] > 0.5): # if low arms correct
                l_num_correct+=1
                break
        
    return [float(o_num_correct) / n, float(f_num_correct) / n, float(b_num_correct) / n, float(s_num_correct) / n, float(l_num_correct) / n]



def sparseE2(predictions, truths):
    # returns vector of length 5 with E0 for one type of data processing
    o_num_correct = 0
    f_num_correct = 0
    b_num_correct = 0
    s_num_correct = 0
    l_num_correct = 0
    n = len(predictions)

    for i in range(len(predictions)):
        pred = predictions[i]

        truth = [int(j) for j in truths[i]]

        o_space = np.arange(max(truth[0] - 2, 0), min(truth[0] + 3, 9))
        f_space = np.arange(max(truth[1] + 10 - 2, 10), min(truth[1] + 10 + 3, 19))
        b_space = np.arange(max(truth[2] + 20 - 2, 20), min(truth[2] + 20 + 3, 29))
        s_space = np.arange(max(truth[3] + 30 - 2, 30), min(truth[3] + 30 + 3, 39))
        l_space = np.arange(max(truth[4] + 40 - 2, 40), min(truth[4] + 40 + 3, 49))

        for truth_index in o_space:
            if (pred[truth_index] > 0.5): # if overstride correct
                o_num_correct+=1
                break

        for truth_index in f_space:
            if (pred[truth_index] > 0.5): # if forward lean correct
                f_num_correct+=1
                break
        
        for truth_index in b_space:
            if (pred[truth_index] > 0.5): # if backward lean correct
                b_num_correct+=1
                break

        for truth_index in s_space:
            if (pred[truth_index] > 0.5): # if sweeping correct
                s_num_correct+=1
                break

        for truth_index in l_space:
            if (pred[truth_index] > 0.5): # if low arms correct
                l_num_correct+=1
                break
        
    return [float(o_num_correct) / n, float(f_num_correct) / n, float(b_num_correct) / n, float(s_num_correct) / n, float(l_num_correct) / n]



def sparseE3(predictions, truths):
    # returns vector of length 5 with E0 for one type of data processing
    o_num_correct = 0
    f_num_correct = 0
    b_num_correct = 0
    s_num_correct = 0
    l_num_correct = 0
    n = len(predictions)

    for i in range(len(predictions)):
        pred = predictions[i]
        
        truth = [int(j) for j in truths[i]]

        o_space = np.arange(max(truth[0] - 3, 0), min(truth[0] + 4, 9))
        f_space = np.arange(max(truth[1] + 10 - 3, 10), min(truth[1] + 10 + 4, 19))
        b_space = np.arange(max(truth[2] + 20 - 3, 20), min(truth[2] + 20 + 4, 29))
        s_space = np.arange(max(truth[3] + 30 - 3, 30), min(truth[3] + 30 + 4, 39))
        l_space = np.arange(max(truth[4] + 40 - 3, 40), min(truth[4] + 40 + 4, 49))

        for truth_index in o_space:
            if (pred[truth_index] > 0.5): # if overstride correct
                o_num_correct+=1
                break

        for truth_index in f_space:
            if (pred[truth_index] > 0.5): # if forward lean correct
                f_num_correct+=1
                break
        
        for truth_index in b_space:
            if (pred[truth_index] > 0.5): # if backward lean correct
                b_num_correct+=1
                break

        for truth_index in s_space:
            if (pred[truth_index] > 0.5): # if sweeping correct
                s_num_correct+=1
                break

        for truth_index in l_space:
            if (pred[truth_index] > 0.5): # if low arms correct
                l_num_correct+=1
                break
        
    return [float(o_num_correct) / n, float(f_num_correct) / n, float(b_num_correct) / n, float(s_num_correct) / n, float(l_num_correct) / n]

In [ ]:
# loading data

raw_X_train_list = []
smoothed_X_train_list = []
trans_X_train_list = []
final_X_train_list = []

y_train_list = []

raw_X_test_list = []
smoothed_X_test_list = []
trans_X_test_list = []
final_X_test_list = []

y_test_list = []
y_sparse_test_list = []

with open("..\\test_examples.txt") as t:
    test_examples = t.readlines()

for example in test_examples:
    raw_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\1_unprocessed\\" + example[:-1])
    smoothed_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\2_smoothed\\" + example[:-1])
    trans_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\3_translation\\" + example[:-1])
    final_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\4_final\\" + example[:-1])

    raw_X_test_list.append(unroll(raw_series))
    smoothed_X_test_list.append(unroll(smoothed_series))
    trans_X_test_list.append(unroll(trans_series))
    final_X_test_list.append(unroll(final_series))
    
    y_test_list.append([int(example[4]), int(example[7]), int(example[10]), int(example[13]), int(example[16])])
    y_sparse_test_list.append(sparse_output(example))

raw_X_test = torch.tensor(raw_X_test_list)
smoothed_X_test = torch.tensor(smoothed_X_test_list)
trans_X_test = torch.tensor(trans_X_test_list)
final_X_test = torch.tensor(final_X_test_list)

y_sparse_test = torch.tensor(y_sparse_test_list)
y_test = torch.tensor(y_test_list)

print(raw_X_test.size())
print(smoothed_X_test.size())
print(trans_X_test.size())
print(final_X_test.size())
print(y_sparse_test.size())
print(y_test.size())
print('\n\n')

with open("..\\training_examples.txt") as t:
    training_examples = t.readlines()

for example in training_examples:
    raw_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\1_unprocessed\\" + example[:-1])
    smoothed_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\2_smoothed\\" + example[:-1])
    trans_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\3_translation\\" + example[:-1])
    final_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\4_final\\" + example[:-1])
    y_example = sparse_output(example)

    raw_X_train_list.append(unroll(raw_series))
    smoothed_X_train_list.append(unroll(smoothed_series))
    trans_X_train_list.append(unroll(trans_series))
    final_X_train_list.append(unroll(final_series))
    
    y_train_list.append(y_example)

raw_X_train = torch.tensor(raw_X_train_list)
smoothed_X_train = torch.tensor(smoothed_X_train_list)
trans_X_train = torch.tensor(trans_X_train_list)
final_X_train = torch.tensor(final_X_train_list)

y_train = torch.tensor(y_train_list)

print(raw_X_train.size())
print(smoothed_X_train.size())
print(trans_X_train.size())
print(final_X_train.size())
print(y_train.size())

In [ ]:
# defining model, parameters, and weight initialization

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)

input_size = 224
hidden_layer_size_1 = 450
hidden_layer_size_2 = 420
hidden_layer_size_3 = 350
hidden_layer_size_4 = 270
hidden_layer_size_5 = 150
hidden_layer_size_6 = 100
output_size = 50
batch_size = 371
learning_rate = 0.00005

model = nn.Sequential(nn.Linear(input_size, hidden_layer_size_1),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_1, hidden_layer_size_2),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_2, hidden_layer_size_3),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_3, hidden_layer_size_4),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_4, hidden_layer_size_5),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_5, hidden_layer_size_6),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_6, output_size),
                        nn.Sigmoid())

loss_function = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

model.apply(init_weights)

num_epochs = 300

In [ ]:
# training and evaluating raw series

min_loss = 1.0

training_cycles = 20

raw_E0 = []
raw_E1 = []
raw_E2 = []
raw_E3 = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(raw_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(raw_X_test)
        test_loss = loss_function(test_y, y_sparse_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (min(test_losses) < min_loss):
        min_loss = min(test_losses)
        test_y = model(raw_X_test)

        raw_E0.clear()
        raw_E1.clear()
        raw_E2.clear()
        raw_E3.clear()

        raw_E0 = sparseE0(test_y, y_test.tolist())
        raw_E1 = sparseE1(test_y, y_test.tolist())
        raw_E2 = sparseE2(test_y, y_test.tolist())
        raw_E3 = sparseE3(test_y, y_test.tolist())

print('MSE: ' + str(min_loss))
print('RAW E0: ' + str(raw_E0))
print('RAW E1: ' + str(raw_E1))
print('RAW E2: ' + str(raw_E2))
print('RAW E3: ' + str(raw_E3))

In [ ]:
# training and evaluating smoothed series

min_loss = 1.0

training_cycles = 20

smooth_E0 = []
smooth_E1 = []
smooth_E2 = []
smooth_E3 = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(smoothed_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(smoothed_X_test)
        test_loss = loss_function(test_y, y_sparse_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (min(test_losses) < min_loss):
        min_loss = min(test_losses)
        test_y = model(smoothed_X_test)

        smooth_E0.clear()
        smooth_E1.clear()
        smooth_E2.clear()
        smooth_E3.clear()

        smooth_E0 = sparseE0(test_y, y_test.tolist())
        smooth_E1 = sparseE1(test_y, y_test.tolist())
        smooth_E2 = sparseE2(test_y, y_test.tolist())
        smooth_E3 = sparseE3(test_y, y_test.tolist())

print('MSE: ' + str(min_loss))
print('SMOOTHED E0: ' + str(smooth_E0))
print('SMOOTHED E1: ' + str(smooth_E1))
print('SMOOTHED E2: ' + str(smooth_E2))
print('SMOOTHED E3: ' + str(smooth_E3))

In [ ]:
# training and evaluating translationally normalized series

min_loss = 1.0

training_cycles = 20

trans_E0 = []
trans_E1 = []
trans_E2 = []
trans_E3 = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(trans_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(trans_X_test)
        test_loss = loss_function(test_y, y_sparse_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (min(test_losses) < min_loss):
        min_loss = min(test_losses)
        test_y = model(trans_X_test)

        trans_E0.clear()
        trans_E1.clear()
        trans_E2.clear()
        trans_E3.clear()

        trans_E0 = sparseE0(test_y, y_test.tolist())
        trans_E1 = sparseE1(test_y, y_test.tolist())
        trans_E2 = sparseE2(test_y, y_test.tolist())
        trans_E3 = sparseE3(test_y, y_test.tolist())

print('MSE: ' + str(min_loss))
print('TRANS E0: ' + str(trans_E0))
print('TRANS E1: ' + str(trans_E1))
print('TRANS E2: ' + str(trans_E2))
print('TRANS E3: ' + str(trans_E3))

In [ ]:
# training and evaluating final series

min_loss = 1.0

training_cycles = 20

final_E0 = []
final_E1 = []
final_E2 = []
final_E3 = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(final_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(final_X_test)
        test_loss = loss_function(test_y, y_sparse_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (min(test_losses) < min_loss):
        min_loss = min(test_losses)
        test_y = model(final_X_test)

        final_E0.clear()
        final_E1.clear()
        final_E2.clear()
        final_E3.clear()

        final_E0 = sparseE0(test_y, y_test.tolist())
        final_E1 = sparseE1(test_y, y_test.tolist())
        final_E2 = sparseE2(test_y, y_test.tolist())
        final_E3 = sparseE3(test_y, y_test.tolist())

print('MSE: ' + str(min_loss))
print('FINAL E0: ' + str(final_E0))
print('FINAL E1: ' + str(final_E1))
print('FINAL E2: ' + str(final_E2))
print('FINAL E3: ' + str(final_E3))